In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# 1️⃣ Load dataset
dataset = load_dataset("json", data_files={"train": "test_fine_tune.json", "test": "test_fine_tune.json"}, field="train")

# 2️⃣ Load tokenizer and model
model_name = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 3️⃣ Preprocessing
def preprocess(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding=True)

dataset = dataset.map(preprocess, batched=True)

# 4️⃣ Remove unused columns for Trainer
dataset = dataset.remove_columns(["premise", "hypothesis"])

# 5️⃣ Fine-tuning with Trainer
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    logging_steps=50,
    learning_rate=2e-5,
    save_total_limit=2,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"]
)

trainer.train()

# 6️⃣ Test fine-tuned model with zero-shot pipeline
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer)

test_texts = [
    "I am looking for a product for my child",
    "I want to adopt a cat",
    "I need assistance for a disabled person",
    "I just want something else"
]

candidate_labels = ["child", "handicap", "pet", "other"]

for text in test_texts:
    result = classifier(text, candidate_labels)
    print(f"\nText: {text}")
    print("Predicted class:", result['labels'][0])
    print("Scores:", result['scores'])


c:\Users\emma\Desktop\project\large-project\venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


Device set to use cpu



Text: I am looking for a product for my child
Predicted class: child
Scores: [0.9959827065467834, 0.00175605365075171, 0.001264249556697905, 0.000997065333649516]

Text: I want to adopt a cat
Predicted class: pet
Scores: [0.9968515038490295, 0.001390138640999794, 0.000978102209046483, 0.0007802379550412297]

Text: I need assistance for a disabled person
Predicted class: handicap
Scores: [0.9981399774551392, 0.000879904895555228, 0.0005158438580110669, 0.00046425481559708714]

Text: I just want something else
Predicted class: other
Scores: [0.8058078289031982, 0.1796957403421402, 0.012700246647000313, 0.0017961935373023152]
